In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import sys, os
sys.path.append(os.path.abspath(".."))
sys.path.append(os.path.abspath("../src"))

from src.data.load_data import load_olist_raw
from src.data.preprocess import merge_olist_tables
from src.data.construct_treatment import create_treatment

In [2]:
df_raw = load_olist_raw()
df = merge_olist_tables(df_raw)
df = create_treatment(df)
print(df.shape)
df.head()

Loaded: orders — (99441, 8)
Loaded: order_items — (112650, 7)
Loaded: order_payments — (103886, 5)
Loaded: order_reviews — (99224, 7)
Loaded: products — (32951, 9)
Loaded: sellers — (3095, 4)
Loaded: customers — (99441, 5)
Merged dataset shape: (114092, 26)
(114092, 29)


,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,order_item_id,product_id,...,product_length_cm,product_height_cm,product_width_cm,customer_city,customer_state,seller_city,seller_state,late_delivery,actual_delivery_days,estimated_delivery_days
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18,1.0,87285b34884572647811a353c7ac498a,...,19.0,8.0,13.0,sao paulo,SP,maua,SP,0,8.0,15
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13,1.0,595fac2a385ac33a80bd5114aec74eb8,...,19.0,13.0,19.0,barreiras,BA,belo horizonte,SP,0,13.0,19
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04,1.0,aa4383b373c6aca5d8797843e5594415,...,24.0,19.0,21.0,vianopolis,GO,guariba,SP,0,9.0,26
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15,1.0,d0b61bfb1de832b15ba9d266ca96e5b0,...,30.0,10.0,20.0,sao goncalo do amarante,RN,belo horizonte,MG,0,13.0,26
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26,1.0,65266b2da20d04dbe00c5c2d3bb7859e,...,51.0,15.0,15.0,santo andre,SP,mogi das cruzes,SP,0,2.0,12


# 1. Causal Problem Definition

We investigate the causal effect of **delivery delays** on **customer satisfaction** using the Olist e-commerce dataset.

Formally, we want to estimate:

$$
\text{ATE} = \mathbb{E}[Y(1) - Y(0)]
$$

where:

- \(T = 1\): the order was delivered *later than the estimated delivery date*  
- \(T = 0\): delivered on time or earlier  
- \(Y\): customer review score (1–5 stars)

---

# 2. Treatment Variable (T)

We define the binary treatment indicator:

### **Late Delivery Indicator**

$$
T = \mathbf{1}(\text{actual delivery date} > \text{estimated delivery date})
$$

In the dataset:

late_delivery = 1 if order_delivered_customer_date > order_estimated_delivery_date
late_delivery = 0 otherwise


Reasons for delays may include long shipping distance, seller performance, product characteristics, peak times, payment verification issues, etc.

---

# 3. Outcome Variable (Y)

We use **customer review score** as the outcome:

- Values range from 1 to 5  
- Higher score = better customer satisfaction  

This variable is a widely used proxy for customer experience.

We can treat the outcome as:

- **continuous** (1–5) — used in this project  

---

# 4. Confounders (X)

Confounders are variables that:

1. Influence the probability of experiencing a late delivery  
2. Also influence the review score  
3. Occur **before** the treatment (to avoid post-treatment bias)

We include features from order metadata, customer location, product characteristics, and seller attributes.

### ✅ Confounders used in the propensity model

**Order-level features**
- purchase day of week  
- purchase hour  
- estimated delivery days  
- price  
- freight value  
- payment type  
- payment value  

**Product attributes**
- product category  
- product volume (height × length × width)  

**Customer attributes**
- customer city  
- customer state  

**Seller attributes**
- seller city  
- seller state  

These confounders influence:

- shipment distance  
- shipping complexity  
- product handling difficulty  
- customer expectations  
- seller reliability  

Thus, they must be controlled for to estimate the causal effect of delivery delays on review scores.

---

# DAG

**Edges:**
- customer_loc → late_delivery (T), customer_loc → review_score (Y)
- product_feat → T, product_feat → Y
- seller_loc → T, seller_loc → Y
- purchase_time → T, purchase_time → Y
- payment_value → T, payment_value → Y
- T → Y



Assumptions:

- No unobserved confounders affecting both T and Y  
- No post-treatment variables included  
- SUTVA (no interference and stable treatment definition)  

This structure is consistent with observational causal inference best practices.

---

# 6. Final Dataset Construction

We construct:

- **T**: `late_delivery`  
- **Y**: `review_score`  
- **X**: selected confounders  

We save the modeling-ready dataset as:

data/processed/model_df.csv


This dataset is used in:

- `03_propensity_model.ipynb` — Logistic regression & XGBoost propensity modeling  
- `04_ipw_aipw_estimation.ipynb` — ATE estimation via IPW/AIPW  
- `05_causal_forest_uplift.ipynb` — Heterogeneous effect estimation  

---


In [3]:
# Convert timestamps
df['order_purchase_dayofweek'] = df['order_purchase_timestamp'].dt.dayofweek
df['order_purchase_hour'] = df['order_purchase_timestamp'].dt.hour

# Product volume
df['product_volume_cm3'] = (
    df['product_height_cm'] * df['product_length_cm'] * df['product_width_cm']
)

# Binary high-value flag
df['is_high_price'] = (df['price'] > df['price'].median()).astype(int)

confounders = [
    'order_purchase_dayofweek',
    'order_purchase_hour',
    'estimated_delivery_days',
    'price',
    'freight_value',
    'product_volume_cm3',
    'payment_value',
    'payment_type',
    'customer_state',
    'customer_city',
    'seller_state',
    'seller_city',
    'product_category_name',
]


In [5]:
model_df = df[["late_delivery", "review_score"] + confounders].dropna()
model_df.to_csv("../data/processed/model_df.csv", index=False)
model_df.head()

,late_delivery,review_score,order_purchase_dayofweek,order_purchase_hour,estimated_delivery_days,price,freight_value,product_volume_cm3,payment_value,payment_type,customer_state,customer_city,seller_state,seller_city,product_category_name
0,0,4.0,0,10,15,29.99,8.72,1976.0,38.71,credit_card,SP,sao paulo,SP,maua,utilidades_domesticas
1,0,4.0,1,20,19,118.70,22.76,4693.0,141.46,boleto,BA,barreiras,SP,belo horizonte,perfumaria
2,0,5.0,2,8,26,159.90,19.22,9576.0,179.12,credit_card,GO,vianopolis,SP,guariba,automotivo
3,0,5.0,5,19,26,45.00,27.20,6000.0,72.20,credit_card,RN,sao goncalo do amarante,MG,belo horizonte,pet_shop
4,0,5.0,1,21,12,19.90,8.72,11475.0,28.62,credit_card,SP,santo andre,SP,mogi das cruzes,papelaria
